In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from PIL import ImageFilter
from tqdm import tqdm
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Flatten, Input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from sklearn.preprocessing import MultiLabelBinarizer
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
import io
import math
import keras.backend.tensorflow_backend as tfb
from keras.regularizers import l2
from keras.utils.generic_utils import CustomObjectScope

%matplotlib inline

tf.__version__

Using TensorFlow backend.


'2.0.0'

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

POS_WEIGHT = 2  # multiplier for positive targets, needs to be tuned

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1192820396111302635
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 17173827973590341676
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 412690511619247785
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10990990132
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6487338382493852660
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:0a:00.0, compute capability: 6.1"
]


In [3]:
data_dir = '/home/kdemochkin/yelp_dataset/'

# Get photo->business mapping from the file provided
train_photo_to_biz_ids = pd.read_csv(data_dir + 'train_photo_to_biz_ids.csv').dropna()
print(train_photo_to_biz_ids)

        photo_id  business_id
0         204149         3034
1          52779         2805
2         278973          485
3         195284          485
4          19992          485
5          80748          485
6         444996         1783
7         200285           35
8          90572           35
9          27565         1313
10        228371         1313
11        166925         2787
12        310868         3216
13         13318         3385
14        290904         2896
15         71940         1193
16        211449         1193
17        368909         1193
18        118126         1193
19        188137         3703
20         12355         3703
21        232115         3703
22        208176         3703
23        362562         3909
24        244500         3909
25         18589          245
26        186937         1605
27         14826         2997
28        156728         3735
29        444034          818
...          ...          ...
234812    329129         3574
234813    

In [4]:
# Get labels for businesses in the training data
train_data_business = pd.read_csv(data_dir + 'train.csv').dropna()
print(train_data_business)

      business_id         labels
0            1000  1 2 3 4 5 6 7
1            1001        0 1 6 8
2             100    1 2 4 5 6 7
3            1006      1 2 4 5 6
4            1010          0 6 8
5             101    1 2 3 4 5 6
6            1011        2 3 5 6
7            1012      1 2 3 5 6
8            1014      1 2 4 5 6
9            1015        1 5 6 7
10           1017          5 6 8
11           1022      0 2 3 5 8
12           1024  0 1 2 3 4 5 6
13           1026    1 2 4 5 6 7
14           1029    1 2 3 5 6 7
15           1030        0 3 5 8
16            103  1 2 3 4 5 6 7
17           1031            6 8
18           1032    1 2 3 5 6 7
19           1035          5 6 8
20           1036    1 2 4 5 6 7
21           1038    1 2 4 5 6 7
22           1039            0 8
23           1040      0 2 6 7 8
24           1041            0 8
25           1043          0 6 8
26            104        0 1 6 8
27           1044            0 8
28           1046          0 6 8
29        

In [9]:
labels = []
photos = []
for photo_id in tqdm(train_photo_to_biz_ids['photo_id'].unique()):
    photos.append(data_dir + 'train_photos/' + str(photo_id) + '.jpg')
    business_id = (train_photo_to_biz_ids[train_photo_to_biz_ids['photo_id'] == photo_id])['business_id'].item()
    if len((train_data_business[train_data_business['business_id'] == business_id])['labels'].values)>0:
        labels.append((train_data_business[train_data_business['business_id'] == business_id])['labels'].values[0].split())
    else:
        labels.append([])

100%|██████████| 234842/234842 [05:27<00:00, 716.14it/s]


In [5]:
def format_example(img):
    img = tf.io.read_file(img)
    img = decode_img(img)
    img = preprocess_input(img)
    # print(img.shape)
    return img

def decode_img(img):
    # convert the compressed string to a 3D uint8 tensor
    img = tf.image.decode_jpeg(img, channels=3)
    # Use `convert_image_dtype` to convert to floats in the [0,1] range.
    # img = tf.image.convert_image_dtype(img, tf.float32)
    # resize the image to the desired size.
    return tf.image.resize(img, [224, 224])

def encode_labels(y):
    one_hot = MultiLabelBinarizer()
    return one_hot.fit_transform(y)

def format_example_2(img):
    img = tf.expand_dims(img, 0) 
    return img

In [10]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 16

encoded_labels = encode_labels(labels)

x_train = tf.data.Dataset.from_tensor_slices(photos[:math.floor(len(photos)*0.9)])
y_train = tf.data.Dataset.from_tensor_slices(encoded_labels[:math.floor(len(encoded_labels)*0.9)])

x_test = tf.data.Dataset.from_tensor_slices(photos[math.floor(len(photos)*0.9):])
y_test = tf.data.Dataset.from_tensor_slices(encoded_labels[math.floor(len(encoded_labels)*0.9):])

x_train = x_train.map(format_example, num_parallel_calls=AUTOTUNE)
x_test = x_test.map(format_example, num_parallel_calls=AUTOTUNE)

train_ds = tf.data.Dataset.zip((x_train, y_train))
test_ds = tf.data.Dataset.zip((x_test, y_test))

train_ds = train_ds.shuffle(buffer_size=512)
train_ds = train_ds.batch(BATCH_SIZE)
train_ds = train_ds.repeat()
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)


test_ds = test_ds.shuffle(buffer_size=512)
test_ds = test_ds.batch(BATCH_SIZE)
test_ds = test_ds.repeat()
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)

train_steps_per_epoch = math.ceil(math.floor(len(photos)*0.9) / BATCH_SIZE)
eval_steps_per_epoch = math.ceil(math.floor(len(photos)*0.1) / BATCH_SIZE)

print(train_steps_per_epoch, eval_steps_per_epoch)

test_ds = iter(test_ds)
train_ds = iter(train_ds)

x_test = x_test.map(format_example_2, num_parallel_calls=AUTOTUNE)

13210 1468


In [6]:
def weighted_binary_crossentropy(target, output):
    """
    Weighted binary crossentropy between an output tensor
    and a target tensor. POS_WEIGHT is used as a multiplier
    for the positive targets.

    Combination of the following functions:
    * keras.losses.binary_crossentropy
    * keras.backend.tensorflow_backend.binary_crossentropy
    * tf.nn.weighted_cross_entropy_with_logits
    """
    # transform back to logits
    _epsilon = tfb._to_tensor(tfb.epsilon(), np.float32)
    output = tf.clip_by_value(output, _epsilon, 1 - _epsilon)
    output =tf.math.log(output / (1 - output))
    # compute weighted loss
    loss = tf.nn.weighted_cross_entropy_with_logits(labels=target, logits=output, pos_weight=POS_WEIGHT)
    return tf.reduce_mean(loss)

In [7]:
def FPR_accuracy(model, y, x):
    val_predict = (np.asarray(model.predict(x))).round()
    val_targ = y
    _val_f1 = f1_score(val_targ, val_predict, average='weighted')
    _val_recall = recall_score(val_targ, val_predict, average='weighted')
    _val_precision = precision_score(val_targ, val_predict, average='weighted')
    val_f1s.append(_val_f1)
    val_recalls.append(_val_recall)
    val_precisions.append(_val_precision)
    print(" — val_f1: %f — val_precision: %f — val_recall %f" %(_val_f1, _val_precision, _val_recall))
    return

# Mobilenet

In [8]:
IM_WIDTH, IM_HEIGHT = 224, 224
NUM_CLASSES = 9
NUM_LAYERS_TO_FREEZE = 22  # 3 conv blocks

top_layers_checkpoint_path = 'result/checkpoint_top_best.hdf5'
fine_tuned_checkpoint_path = 'result/checkpoint_fine_tuned_best.hdf5'
new_extended_mobilenet_weights = 'result/final_weights.hdf5'

print('building model')
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(IM_WIDTH, IM_HEIGHT, 3))

top_layer = base_model.output
top_layer = GlobalAveragePooling2D()(top_layer)
top_layer = Dropout(rate=0.5)(top_layer)
predictions = Dense(NUM_CLASSES, activation='sigmoid')(top_layer)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False
    
model.summary()

optimizer=Adam(lr=0.001, decay=0.2)

building model
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 

In [11]:
epochs = 3

print('transfer learning')
for epoch in range(epochs):
    print("Epoch ", str(epoch))
    for step in tqdm(range(train_steps_per_epoch)):
        x_batch_train, y_batch_train = next(train_ds)
        with tf.GradientTape() as tape:
            logits = model(x_batch_train)
            loss_value = weighted_binary_crossentropy(tf.cast(y_batch_train, dtype=tf.float32), 
                                                      tf.cast(logits, dtype=tf.float32))
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        if step % 1000 == 0:
            print('Training loss (for one batch) at step %s: %s' % (step, float(loss_value)))

    y_pred = []
    y_true = []
    for eval_step in tqdm(range(eval_steps_per_epoch)):
        x_batch_val, y_batch_val = next(test_ds)
        y_pred.append(np.where(model(x_batch_val).numpy() > 0.5, 1, 0))
        y_true.append(y_batch_val.numpy())
        
    y_pred = np.concatenate(y_pred)
    y_true = np.concatenate(y_true)
    
    _val_f1 = f1_score(y_true, y_pred, average='micro')
    _val_recall = recall_score(y_true, y_pred, average='micro')
    _val_precision = precision_score(y_true, y_pred, average='micro')
        
    print('Validation acc - f1: %f, recall: %f, precision: %f' %(_val_f1, _val_recall, _val_precision))

  0%|          | 0/13210 [00:00<?, ?it/s]

transfer learning
Epoch  0


  0%|          | 3/13210 [00:07<19:24:59,  5.29s/it]

Training loss (for one batch) at step 0: 1.4251139163970947


  8%|▊         | 1003/13210 [01:36<16:06, 12.63it/s]

...

Training loss (for one batch) at step 12000: 0.8255971074104309


 98%|█████████▊| 13002/13210 [17:04<00:16, 12.73it/s]

Training loss (for one batch) at step 13000: 0.9047096967697144


100%|██████████| 1468/1468 [00:41<00:00, 33.61it/s]


Validation acc - f1: 0.719717, recall: 0.872410, precision: 0.612513


In [12]:
model.save("results/transfer_learning.h5")

In [13]:
# fine tuning
print('setup finetuning')

for layer in base_model.layers:
    layer.trainable = True
    
model.summary()

setup finetuning
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112

In [14]:
print('finetune train')
epochs = 3
for epoch in range(epochs):
    print("Epoch ", str(epoch))
    for step in tqdm(range(train_steps_per_epoch)):
        x_batch_train, y_batch_train = next(train_ds)
        with tf.GradientTape() as tape:
            logits = model(x_batch_train)
            loss_value = weighted_binary_crossentropy(tf.cast(y_batch_train, dtype=tf.float32), 
                                                      tf.cast(logits, dtype=tf.float32))
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        if step % 1000 == 0:
            print('Training loss (for one batch) at step %s: %s' % (step, float(loss_value)))

    y_pred = []
    y_true = []
    for eval_step in tqdm(range(eval_steps_per_epoch)):
        x_batch_val, y_batch_val = next(test_ds)
        y_pred.append(np.where(model(x_batch_val).numpy() > 0.5, 1, 0))
        y_true.append(y_batch_val.numpy())
        
    y_pred = np.concatenate(y_pred)
    y_true = np.concatenate(y_true)
    
    _val_f1 = f1_score(y_true, y_pred, average='micro')
    _val_recall = recall_score(y_true, y_pred, average='micro')
    _val_precision = precision_score(y_true, y_pred, average='micro')
        
    print('Validation acc - f1: %f, recall: %f, precision: %f' %(_val_f1, _val_recall, _val_precision))

  0%|          | 0/13210 [00:00<?, ?it/s]

finetune train
Epoch  0


  0%|          | 3/13210 [00:00<49:51,  4.41it/s]  

Training loss (for one batch) at step 0: 0.7116513252258301


  8%|▊         | 1003/13210 [01:21<16:27, 12.36it/s]

...

Training loss (for one batch) at step 12000: 0.8103947043418884


 98%|█████████▊| 13002/13210 [17:35<00:16, 12.26it/s]

Training loss (for one batch) at step 13000: 0.6898121237754822


100%|██████████| 1468/1468 [00:41<00:00, 33.74it/s]


Validation acc - f1: 0.759912, recall: 0.906807, precision: 0.653974


In [15]:
print('saving')
model.save('results/final_weights.hdf5')

saving


# Inception

In [16]:
IM_WIDTH, IM_HEIGHT = 224, 224
NUM_CLASSES = 9
NUM_LAYERS_TO_FREEZE = 172  # 3 conv blocks

top_layers_checkpoint_path = 'result/checkpoint_top_best.hdf5'
fine_tuned_checkpoint_path = 'result/checkpoint_fine_tuned_best.hdf5'
new_extended_mobilenet_weights = 'result/final_weights.hdf5'

print('building model')
base_model = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, 
                                               input_shape=(IM_WIDTH, IM_HEIGHT, 3))

top_layer = base_model.output
top_layer = GlobalAveragePooling2D()(top_layer)
top_layer = Dropout(rate=0.5)(top_layer)
predictions = Dense(NUM_CLASSES, activation='sigmoid')(top_layer)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False
    
model.summary()

optimizer=Adam(lr=0.001, decay=0.2)

building model
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
_____________________________________________________________________________

In [17]:
epochs = 3

print('transfer learning')
for epoch in range(epochs):
    print("Epoch ", str(epoch))
    for step in tqdm(range(train_steps_per_epoch)):
        x_batch_train, y_batch_train = next(train_ds)
        with tf.GradientTape() as tape:
            logits = model(x_batch_train)
            loss_value = weighted_binary_crossentropy(tf.cast(y_batch_train, dtype=tf.float32), 
                                                      tf.cast(logits, dtype=tf.float32))
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        if step % 1000 == 0:
            print('Training loss (for one batch) at step %s: %s' % (step, float(loss_value)))

    y_pred = []
    y_true = []
    for eval_step in tqdm(range(eval_steps_per_epoch)):
        x_batch_val, y_batch_val = next(test_ds)
        y_pred.append(np.where(model(x_batch_val).numpy() > 0.5, 1, 0))
        y_true.append(y_batch_val.numpy())
        
    y_pred = np.concatenate(y_pred)
    y_true = np.concatenate(y_true)
    
    _val_f1 = f1_score(y_true, y_pred, average='micro')
    _val_recall = recall_score(y_true, y_pred, average='micro')
    _val_precision = precision_score(y_true, y_pred, average='micro')
        
    print('Validation acc - f1: %f, recall: %f, precision: %f' %(_val_f1, _val_recall, _val_precision))

  0%|          | 0/13210 [00:00<?, ?it/s]

transfer learning
Epoch  0


  0%|          | 2/13210 [00:00<1:28:29,  2.49it/s]

Training loss (for one batch) at step 0: 1.5629686117172241


  8%|▊         | 1002/13210 [02:26<30:10,  6.74it/s]

...

Training loss (for one batch) at step 12000: 0.8063651919364929


 98%|█████████▊| 13002/13210 [31:42<00:30,  6.91it/s]

Training loss (for one batch) at step 13000: 0.8337997794151306


100%|██████████| 1468/1468 [01:53<00:00, 12.08it/s]

Validation acc - f1: 0.725205, recall: 0.888268, precision: 0.612724


In [18]:
model.save("results/transfer_learning_inception.h5")

In [19]:
# fine tuning
print('setup finetuning')

for layer in model.layers:
    layer.trainable = True
    
model.summary()

setup finetuning
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
___________________________________________________________________________

In [20]:
print('finetune train')
epochs = 3
for epoch in range(epochs):
    print("Epoch ", str(epoch))
    for step in tqdm(range(train_steps_per_epoch)):
        x_batch_train, y_batch_train = next(train_ds)
        with tf.GradientTape() as tape:
            logits = model(x_batch_train)
            loss_value = weighted_binary_crossentropy(tf.cast(y_batch_train, dtype=tf.float32), 
                                                      tf.cast(logits, dtype=tf.float32))
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        if step % 1000 == 0:
            print('Training loss (for one batch) at step %s: %s' % (step, float(loss_value)))

    y_pred = []
    y_true = []
    for eval_step in tqdm(range(eval_steps_per_epoch)):
        x_batch_val, y_batch_val = next(test_ds)
        y_pred.append(np.where(model(x_batch_val).numpy() > 0.5, 1, 0))
        y_true.append(y_batch_val.numpy())
        
    y_pred = np.concatenate(y_pred)
    y_true = np.concatenate(y_true)
    
    _val_f1 = f1_score(y_true, y_pred, average='micro')
    _val_recall = recall_score(y_true, y_pred, average='micro')
    _val_precision = precision_score(y_true, y_pred, average='micro')
        
    print('Validation acc - f1: %f, recall: %f, precision: %f' %(_val_f1, _val_recall, _val_precision))

  0%|          | 0/13210 [00:00<?, ?it/s]

finetune train
Epoch  0


  0%|          | 2/13210 [00:00<1:48:31,  2.03it/s]

Training loss (for one batch) at step 0: 0.72471022605896


  8%|▊         | 1002/13210 [02:56<35:02,  5.81it/s]

...

Training loss (for one batch) at step 12000: 0.8168604969978333


 98%|█████████▊| 13002/13210 [38:14<00:36,  5.74it/s]

Training loss (for one batch) at step 13000: 0.7207087278366089


100%|██████████| 1468/1468 [01:52<00:00, 12.01it/s]

Validation acc - f1: 0.770059, recall: 0.906896, precision: 0.669101


In [21]:
print('saving')
model.save('results/final_weights_inception.hdf5')

saving
